In [44]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as fn
import torchvision
import cv2
import time
from matplotlib import pyplot as plt
from torchsummary import summary


device = 'cuda:0' if torch.cuda.is_available() else 'cpu'


In [45]:
class View(nn.Module):
    
    def __init__(self, *shape): 
        super(View, self).__init__() 
        self.shape = shape
        
    def forward(self, x):
        return x.view(x.shape[0], *self.shape) # x.shape = [batch_size, channel, width, height]

In [46]:
class BottleNeck(nn.Module):
    multiple = 4
    def __init__(self, n_channel):
        super(BottleNeck, self).__init__()
        layers = []
        layers += [nn.Conv2d(in_channels = n_channel, out_channels = n_channel, kernel_size = 1,stride = 1, padding = 1),
                   nn.BatchNorm2d(num_features = n_channel),
                   nn.ReLU(),
                   nn.Conv2d(in_channels = n_channel, out_channels = n_channel, kernel_size = 3,stride = 1, padding = 1),
                   nn.BatchNorm2d(num_features = n_channel),
                   nn.ReLU(),
                   nn.Conv2d(in_channels = n_channel, out_channels = n_channel*BottleNeck.multiple, kernel_size = 1,stride = 1, padding = 3),
                   nn.BatchNorm2d(num_features = n_channel * BottleNeck.multiple),
                   nn.ReLU()]
        self.shortcut = nn.Sequential(*layers)
    def forward(self, x):
        out = self.layers(x)
        return x + out

In [53]:
class Resnet(nn.Module):
    def __init__(self):
        super(Resnet, self).__init__()

        layers = []
        layers += [nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size = 7, stride = 2, padding = 3),
                nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1),
                nn.BatchNorm2d(num_features = 64),
                nn.ReLU(),
                BottleNeck(n_channel=64),
                BottleNeck(n_channel=64),
                BottleNeck(n_channel=64),
                BottleNeck(n_channel=128),
                BottleNeck(n_channel=128),
                BottleNeck(n_channel=128),
                BottleNeck(n_channel=256),
                BottleNeck(n_channel=256),
                BottleNeck(n_channel=256),
                BottleNeck(n_channel=512),
                BottleNeck(n_channel=512),
                BottleNeck(n_channel=512),
                nn.AdaptiveAvgPool2d((1,1)), # batch * 256 * 1 * 1로 텐서 변환 되게
                View(-1),
                nn.Linear(in_features = 2048, out_features = 1000)
                ]
    def forward(self,x):
        return self.layers(x)
        

In [54]:
model = Resnet()

In [55]:
if __name__ == '__main__': # 이 파일이 메인으로 돌아갈때만 아래를 실행한다
    model = Resnet()
    summary(model, (1,112,112))

AttributeError: 'Resnet' object has no attribute 'layers'